In [ ]:
import os
import logging
import json
import requests
import pandas as pd
from typing import Any, Callable, Set
from azure.identity import AzureCliCredential  # or DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FunctionTool, ToolSet
from azure.keyvault.secrets import SecretClient
from atproto import Client as BskyClient

azure_logger = logging.getLogger("azure")
azure_logger.setLevel(logging.WARNING)  # Set to WARNING, ERROR, or CRITICAL to reduce logs

# Optionally, suppress other verbose logs
logging.getLogger().setLevel(logging.WARNING)

# --------------------------------------------------------------------
# 1) Load Environment Variables (Locally) or from Key Vault
# --------------------------------------------------------------------
# If you're storing secrets in .env locally, do:

keyvault_name = "<your-keyvault-name>"
kv_client = SecretClient(f"https://{keyvault_name}.vault.azure.net/", AzureCliCredential())

# Pull secrets from environment variables
try:
    PROJECT_CONNECTION_STRING = kv_client.get_secret("project-connection-string").value
    AZURE_FUNCTIONS_KEY       = kv_client.get_secret("azure-relataly-functions-key").value
    BSKY_USERNAME             = kv_client.get_secret("bskyusername").value
    BSKY_PASSWORD             = kv_client.get_secret("bskypw").value
    BING_API_KEY              = kv_client.get_secret("bing-search-api").value
except Exception as e:
     logging.error(f"Error retrieving secrets from Key Vault: {e}")
     raise
 
def retrieve_secret(secret_name: str, kv_client, suppress_logs: bool = False) -> str:
    """
    Retrieves a secret from Azure Key Vault, with an option to suppress logging.
    """
    azure_logger = logging.getLogger("azure")
    original_level = azure_logger.level
    if suppress_logs:
        azure_logger.setLevel(logging.CRITICAL)  # Temporarily suppress logs
    try:
        secret_value = kv_client.get_secret(secret_name).value
        return secret_value
    except Exception as e:
        logging.error(f"Error retrieving secret '{secret_name}' from Key Vault: {e}")
        raise
    finally:
        if suppress_logs:
            azure_logger.setLevel(original_level)  # Restore original logging level


logging.basicConfig(level=logging.INFO)

# --------------------------------------------------------------------
# 2) Helper Functions
# --------------------------------------------------------------------
def create_tiny_url(url: str) -> str:
    """
    Shortens a given URL using TinyURL API.
    """
    try:
        response = requests.get(f'http://tinyurl.com/api-create.php?url={url}')
        return response.text
    except Exception as e:
        logging.error(f"TinyURL error: {e}")
        return url

def check_tweet_length(tweet_text: str) -> str:
    """
    Checks if the tweet text is within the 280-character limit.
    Returns a JSON string with message & length.
    """
    logging.warning("Agent checks tweet length...")
    
    if len(tweet_text) <= 280:
        message = "Tweet is within the character limit."
        logging.warning("Tweet is within the character limit." + " LENGTH: " + str(len(tweet_text)))
    else:
        message = "Tweet is too long. Please shorten it."
        logging.warning("Tweet is too long. Please shorten it." + " LENGTH: " + str(len(tweet_text)))
    return json.dumps({"message": message, "length": len(tweet_text)})

def search_for_relevant_news_via_bingsearch(query: str, news_count: int = 10) -> str:
    """
    Searches for relevant news using the Bing Search API.
    Returns a JSON string of the news data or error.
    """
    logging.warning("Agent searches for relevant news via Bing Search... with keyword: " + query)
    
    endpoint = "https://api.bing.microsoft.com/v7.0/news/search"
    #query = "Artificial Intelligence"
    params = {'q': query, 'mkt': 'en-US', 'count': news_count}
    headers = {'Ocp-Apim-Subscription-Key': BING_API_KEY}
    try:
        response = requests.get(endpoint, headers=headers, params=params)
        response.raise_for_status()
        return json.dumps(response.json())
    except Exception as e:
        logging.error(f"Bing Search error: {e}")
        return json.dumps({"error": str(e)})

def receive_previous_posts_from_bluesky_social_media() -> str:
    """
    Retrieves the latest posts from Bluesky for a specified actor, including
    like counts, reply counts, timestamps, marking comments explicitly, and
    related post IDs for comments.
    """
    logging.warning("Agent receives previous posts from Bluesky...")
    
    if not BSKY_USERNAME or not BSKY_PASSWORD:
        logging.error("Missing Bluesky credentials.")
        return json.dumps([])

    bskyclient = BskyClient(base_url='https://bsky.social')
    bskyclient.login(BSKY_USERNAME, BSKY_PASSWORD)
    logging.info("Logged into Bluesky.")

    # Example actor ID
    actor_id = "did:plc:mvtnlat6pttm63kxauko7fri"
    data = bskyclient.get_author_feed(actor=actor_id, filter='posts_and_author_threads', limit=10)

    posts = []
    for feed_item in data.feed:
        post_view = getattr(feed_item, 'post', None)
        record = getattr(post_view, 'record', None) if post_view else None
        
        if not post_view or not record:
            continue
        
        # Extract post details
        text = getattr(record, 'text', None)
        created_at = getattr(record, 'created_at', None)  # Post creation time
        like_count = getattr(post_view, 'like_count', 0)  # Default to 0 if not present
        reply_count = getattr(post_view, 'reply_count', 0)  # Default to 0 if not present
        post_id = getattr(post_view, 'cid', None)  # Unique post ID
        related_post_id = None  # Default for non-comments

        # Check if it's a comment and get related post ID
        is_comment = hasattr(record, 'reply') and record.reply is not None
        if is_comment and hasattr(record.reply, 'parent'):
            related_post_id = getattr(record.reply.parent, 'cid', None)

        posts.append({
            "text": text,
            "created_at": created_at,
            "like_count": like_count,
            "reply_count": reply_count,
            "post_id": post_id,
            "is_comment": is_comment,
            "related_post_id": related_post_id
        })
        logging.warning(f"Received the following post from Bluesky: " + text + " CREATED_AT: " + str(created_at))
    
    return json.dumps(posts, indent=4)

def call_function_to_post_on_bluesky_social_media(title: str, tweet_text: str, url: str, link_title: str = "") -> str:
    """
    Posts to Bluesky via an Azure Function (HttpCreateBlueskyTweetRaw).
    Returns the JSON string of the response or error.
    """
    logging.warning("Agent calls function to post on Bluesky... TITLE: " + title + " TEXT: " + tweet_text + " URL: " + url + " LINK_TITLE: " + link_title)
    
    shortened_url = create_tiny_url(url)
    request_url = "https://relatalyfunc.azurewebsites.net/api/HttpCreateBlueskyTweetRaw"
    payload = json.dumps({
        "tweet": tweet_text,
        "url": shortened_url,
        "link_title": link_title
    })
    headers = {
        "x-functions-key": AZURE_FUNCTIONS_KEY,
        "Content-Type": "application/json"
    }

    try:
        response = requests.post(request_url, headers=headers, data=payload)
        return json.dumps({"status": response.status_code, "response": response.text})
    except Exception as e:
        logging.error(f"Error calling Bluesky posting function: {e}")
        return json.dumps({"error": str(e)})

# --------------------------------------------------------------------
# 3) Put all user-defined functions in a FunctionTool
# --------------------------------------------------------------------
user_functions: Set[Callable[..., Any]] = {
    search_for_relevant_news_via_bingsearch,
    check_tweet_length,
    receive_previous_posts_from_bluesky_social_media,
    call_function_to_post_on_bluesky_social_media
}

# --------------------------------------------------------------------
# 4) Run Pipeline (mimics the Azure AI Projects logic)
# --------------------------------------------------------------------
def run_ai_project_pipeline():
    """
    Creates an Azure AI Projects agent, runs a conversation, and cleans up.
    """
    # 1) Create the Azure AI Client from your connection string
    project_client = AIProjectClient.from_connection_string(
        credential=AzureCliCredential(),  # or DefaultAzureCredential()
        conn_str=PROJECT_CONNECTION_STRING
    )

    # 2) Wrap the Python user functions in a FunctionTool
    functions = FunctionTool(user_functions)

    # 3) Put the FunctionTool in a ToolSet
    toolset = ToolSet()
    toolset.add(functions)

    instructions = """
    You are a helpful assistant with the goal to post about relevant AI news on Bluesky social media.
    When a user requests, you will fetch the latest AI news and create a post on Bluesky with a tweet and a link to the news article.
    Follow these steps to ensure accurate and concise responses:

    1. **Fetch News from Bing Search**: Always use the 'search_for_relevant_news_via_bingsearch' function to retrieve any AI related news.
    2. **Get Your Recent Bluesky Feed**: Always use the 'receive_previous_posts_from_bluesky_social_media' function to retrieve the latest posts in order to avoid redundant posts.
    3. **Evaluate the News**: Identify the most interest news article from Bing search results considering previous posts to avoid posting about the same topic twice.
    4. **Create a Tweet**: Create a tweet text about the selected news (avoid topics from previous posts). Always use the 'check_tweet_length' function to ensure the tweet is within the 280-character limit. Avoid adding the url into the text and instead provide the url as part of the call_function_to_post_on_bluesky_social_media function.
    5. **Error Handling**: If there are issues, inform the user about the problem and end the process.

    Remember, clarity and brevity are key. Ensure your responses are direct and informative.
    """

    with project_client:
        # 4) Create and run the agent
        agent = project_client.agents.create_agent(
            model="gpt-4o-mini",
            name="my-agent",
            instructions=instructions,
            toolset=toolset
        )
        logging.info(f"Created agent, ID: {agent.id}")

        # 5) Create a thread for conversation
        thread = project_client.agents.create_thread()
        logging.info(f"Created thread, ID: {thread.id}")

        # 6) Create a user message
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content="Hello! I'd like to post a tweet about a new AI article."
        )
        logging.info(f"Created message, ID: {message.id}")

        # 7) Create and process a run
        run = project_client.agents.create_and_process_run(
            thread_id=thread.id,
            assistant_id=agent.id
        )
        logging.info(f"Run finished with status: {run.status}")

        if run.status == "failed":
            logging.error(f"Run failed: {run.last_error}")

        # 8) List all messages (including the final assistant message)
        messages = project_client.agents.list_messages(thread_id=thread.id)
        logging.info("----- MESSAGES -----")
        for msg in messages["data"]:
            if msg["role"] == "assistant":
                logging.info("Assistant says:")
                for part in msg["content"]:
                    if part["type"] == "text":
                        logging.info(part["text"]["value"])
            elif msg["role"] == "user":
                logging.info(f"User says: {msg['content'][0]['text']['value']}")

        # 9) Cleanup: delete the agent
        project_client.agents.delete_agent(agent.id)
        logging.info("Deleted agent successfully.")

# --------------------------------------------------------------------
# 5) Local Test Entry Point
# --------------------------------------------------------------------
# Simply call run_ai_project_pipeline() in your notebook to test locally.
if __name__ == "__main__":
    run_ai_project_pipeline()
    